In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
full = pd.read_csv('full.csv')
full.head()

,Unnamed: 0,bin,roofarea,svf,numbldgs,assesstot,bldgarea,comarea,yearbuilt_1900_1940,yearbuilt_1940_1980,...,pctLatino,pctUnder18,pctOver62,solar,approved1_block,approved2_block,approved5_block,since_yearalter1,since_yearalter2,popdensity
0,0,3000002.0,17683.475106,0.769877,1.0,17318250.0,170493.0,163894.0,0,0,...,6.4,20.1,3.2,0,0,0,0,27.0,6.0,0.000835
1,180,3413929.0,9431.126565,0.548905,1.0,12383114.0,102534.0,4686.0,0,0,...,6.4,20.1,3.2,0,0,0,0,2021.0,2021.0,0.000835
2,1,3000007.0,8192.772710,0.730845,1.0,371250.0,9585.0,9585.0,0,0,...,6.4,20.1,3.2,0,0,0,0,2021.0,2021.0,0.000835
3,2,3000010.0,43459.105586,0.729904,1.0,30760650.0,500000.0,500000.0,1,0,...,6.4,20.1,3.2,0,0,0,0,22.0,2021.0,0.000835
4,3,3000011.0,19246.260335,0.827548,1.0,1474200.0,21735.0,21735.0,0,1,...,6.4,20.1,3.2,0,0,0,0,9.0,2021.0,0.000835


In [68]:
features = full.columns.difference(['solar','GEOID','Unnamed: 0','bin','since_yearalter1','since_yearalter2'])
X = full[features]
y = full['solar']

In [69]:
from sklearn.cluster import KMeans

In [88]:
k = 20
kmeans = KMeans(n_clusters=k, random_state=42)
y_pred = kmeans.fit_predict(X)

In [89]:
y_pred

array([15,  6,  5, ...,  1, 17, 17], dtype=int32)

In [90]:
full['cluster'] = y_pred

In [91]:
full.head()

,Unnamed: 0,bin,roofarea,svf,numbldgs,assesstot,bldgarea,comarea,yearbuilt_1900_1940,yearbuilt_1940_1980,...,pctUnder18,pctOver62,solar,approved1_block,approved2_block,approved5_block,since_yearalter1,since_yearalter2,popdensity,cluster
0,0,3000002.0,17683.475106,0.769877,1.0,17318250.0,170493.0,163894.0,0,0,...,20.1,3.2,0,0,0,0,27.0,6.0,0.000835,15
1,180,3413929.0,9431.126565,0.548905,1.0,12383114.0,102534.0,4686.0,0,0,...,20.1,3.2,0,0,0,0,2021.0,2021.0,0.000835,6
2,1,3000007.0,8192.772710,0.730845,1.0,371250.0,9585.0,9585.0,0,0,...,20.1,3.2,0,0,0,0,2021.0,2021.0,0.000835,5
3,2,3000010.0,43459.105586,0.729904,1.0,30760650.0,500000.0,500000.0,1,0,...,20.1,3.2,0,0,0,0,22.0,2021.0,0.000835,15
4,3,3000011.0,19246.260335,0.827548,1.0,1474200.0,21735.0,21735.0,0,1,...,20.1,3.2,0,0,0,0,9.0,2021.0,0.000835,5


In [92]:
total_count = len(full)
total_solar = len(full[full.solar == 1])
print(total_count)
print(total_solar)

267634
5204


In [93]:
results = pd.DataFrame(columns=['cluster','percent_total','percent_solar','diff'])
for i in range(20):
#     print('--------------')
#     print(i)
    tot = len(full[full.cluster == i])
    s = len(full[(full.cluster == i) & (full.solar == 1)])
    percent_tot = tot/total_count*100
    percent_s = s/total_solar*100
    results = results.append({
        'cluster':i,
        'percent_total':percent_tot,
        'percent_solar':percent_s,
        'diff': percent_s - percent_tot
    },ignore_index=True)
results

,cluster,percent_total,percent_solar,diff
0,0.0,4.921647,2.056111,-2.865536
1,1.0,0.975212,3.958493,2.983281
2,2.0,0.000374,0.000000,-0.000374
3,3.0,0.229418,0.115296,-0.114122
4,4.0,0.013825,0.000000,-0.013825
5,5.0,1.822265,2.632590,0.810326
6,6.0,0.103126,0.230592,0.127466
7,7.0,0.478639,0.787855,0.309217
8,8.0,40.761264,38.758647,-2.002616
9,9.0,4.062638,3.708686,-0.353952


In [45]:
full[full.cluster == 1]

,Unnamed: 0,bin,roofarea,svf,numbldgs,assesstot,bldgarea,comarea,yearbuilt_1900_1940,yearbuilt_1940_1980,...,pctUnder18,pctOver62,solar,approved1_block,approved2_block,approved5_block,since_yearalter1,since_yearalter2,popdensity,cluster
18158,18198,3398156.0,200331.948441,0.775741,1.0,1.166037e+09,660000.0,660000.0,0,0,...,21.0,9.2,0,0,0,0,12.0,2021.0,0.001753,1


In [21]:
len(full[full.solar == 1])

5204

In [35]:
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt


In [33]:
kmeans_per_k = [KMeans(n_clusters=k, random_state=42).fit(X)
                for k in range(1, 10)]
inertias = [model.inertia_ for model in kmeans_per_k]

In [ ]:
plt.figure(figsize=(8, 3.5))
plt.plot(range(1, 10), inertias, "bo-")
plt.xlabel("$k$", fontsize=14)
plt.ylabel("Inertia", fontsize=14)
plt.annotate('Elbow',
             xy=(4, inertias[3]),
             xytext=(0.55, 0.55),
             textcoords='figure fraction',
             fontsize=16,
             arrowprops=dict(facecolor='black', shrink=0.1)
            )
plt.axis([1, 8.5, 0, 1300])
# save_fig("inertia_vs_k_plot")
plt.show()